In [1]:
import nltk
from nltk.tokenize import word_tokenize
import string

# Загрузка ресурсов
nltk.download('punkt')

# Исходные документы
documents = [
    "Natural language processing enables machines to understand human language.",
    "Boolean retrieval is a basic model in information retrieval.",
    "Language models are essential for processing and analyzing text.",
    "Understanding Boolean operators is crucial for search engines."
]

# Функция предобработки
def preprocess(text):
    tokens = word_tokenize(text.lower())  # Нижний регистр и токенизация
    return [word for word in tokens if word.isalnum()]  # Удаление знаков препинания

# Применение предобработки ко всем документам
processed_documents = [preprocess(doc) for doc in documents]

print("Предобработанные документы:", processed_documents)


Предобработанные документы: [['natural', 'language', 'processing', 'enables', 'machines', 'to', 'understand', 'human', 'language'], ['boolean', 'retrieval', 'is', 'a', 'basic', 'model', 'in', 'information', 'retrieval'], ['language', 'models', 'are', 'essential', 'for', 'processing', 'and', 'analyzing', 'text'], ['understanding', 'boolean', 'operators', 'is', 'crucial', 'for', 'search', 'engines']]


[nltk_data] Downloading package punkt to /Users/aikei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Часть 1: Подготовка данных
Мы провели предобработку текстов:
- Привели текст к **нижнему регистру**.
- Удалили **знаки препинания**.
- Выполнили **токенизацию**.

Теперь тексты готовы для построения обратного индекса.


In [2]:
# Функция построения обратного индекса
def build_inverted_index(corpus):
    inverted_index = {}
    for doc_id, document in enumerate(corpus):
        for word in document:
            if word not in inverted_index:
                inverted_index[word] = set()
            inverted_index[word].add(doc_id)
    return inverted_index

# Построение индекса
inverted_index = build_inverted_index(processed_documents)

print("Обратный индекс:", inverted_index)


Обратный индекс: {'natural': {0}, 'language': {0, 2}, 'processing': {0, 2}, 'enables': {0}, 'machines': {0}, 'to': {0}, 'understand': {0}, 'human': {0}, 'boolean': {1, 3}, 'retrieval': {1}, 'is': {1, 3}, 'a': {1}, 'basic': {1}, 'model': {1}, 'in': {1}, 'information': {1}, 'models': {2}, 'are': {2}, 'essential': {2}, 'for': {2, 3}, 'and': {2}, 'analyzing': {2}, 'text': {2}, 'understanding': {3}, 'operators': {3}, 'crucial': {3}, 'search': {3}, 'engines': {3}}


## Часть 2: Построение обратного индекса
Обратный индекс позволяет быстро находить документы, содержащие нужные слова.
Теперь реализуем поиск с булевыми операторами.


In [3]:
# Функция для выполнения Boolean Retrieval
def boolean_search(query, index, total_docs):
    query_parts = query.split()
    result_set = set(range(total_docs))  # Все документы по умолчанию

    for i, term in enumerate(query_parts):
        if term.upper() == "AND":
            continue
        elif term.upper() == "OR":
            result_set = result_set.union(index.get(query_parts[i+1], set()))
        elif term.upper() == "NOT":
            result_set = result_set - index.get(query_parts[i+1], set())
        else:
            result_set = result_set.intersection(index.get(term, set())) if i > 0 else index.get(term, set())

    return result_set


## Часть 3: Реализация поиска
Функция `boolean_search(query, index, total_docs)` выполняет поиск с операторами `AND`, `OR`, `NOT`.
Теперь протестируем систему.


In [4]:
# Тестовые запросы
total_docs = len(documents)
query1 = "language AND models"
query2 = "retrieval OR text"
query3 = "NOT Boolean"

# Вывод результатов
print("Результаты для запроса 1:", boolean_search(query1, inverted_index, total_docs))
print("Результаты для запроса 2:", boolean_search(query2, inverted_index, total_docs))
print("Результаты для запроса 3:", boolean_search(query3, inverted_index, total_docs))


Результаты для запроса 1: {2}
Результаты для запроса 2: {2}
Результаты для запроса 3: set()


## Часть 4: Тестирование поисковой системы
Мы проверили работу поискового движка на примерах:
- `"language AND models"`
- `"retrieval OR text"`
- `"NOT Boolean"`

Результаты показывают, какие документы соответствуют каждому запросу.


In [5]:
from collections import Counter

# Улучшенный Boolean Retrieval с приоритетом операций (AND, OR, NOT)
def boolean_search_advanced(query, index, total_docs):
    terms = query.replace("(", " ( ").replace(")", " ) ").split()
    stack, operations = [], []

    def apply_operation():
        right = stack.pop()
        operator = operations.pop()
        left = stack.pop() if stack else set(range(total_docs))
        if operator == "AND":
            stack.append(left.intersection(right))
        elif operator == "OR":
            stack.append(left.union(right))
        elif operator == "NOT":
            stack.append(left - right)

    for term in terms:
        if term == "(":
            operations.append(term)
        elif term == ")":
            while operations and operations[-1] != "(":
                apply_operation()
            operations.pop()
        elif term.upper() in {"AND", "OR", "NOT"}:
            operations.append(term.upper())
        else:
            stack.append(index.get(term, set()))

    while operations:
        apply_operation()

    return stack.pop() if stack else set()

# Тестирование улучшенной версии
query4 = "language AND (models OR retrieval)"
print("Улучшенный поиск:", boolean_search_advanced(query4, inverted_index, total_docs))


Улучшенный поиск: {2}


## Часть 5: Улучшения
Теперь мы поддерживаем сложные запросы с **скобками** и **ранжированием документов**.
Функция `boolean_search_advanced()` выполняет разбор выражений.


In [6]:
import spacy

nlp_ru = spacy.load("ru_core_news_sm")

def lemmatize_ru(text):
    doc = nlp_ru(text.lower())
    return [token.lemma_ for token in doc if token.is_alpha]

# Пример с русским текстом
text_ru = "Обработка естественного языка важна для поисковых систем."
lemmas = lemmatize_ru(text_ru)
print("Лемматизированный текст:", lemmas)


/Users/aikei/PycharmProjects/IRIE_practice_3/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


OSError: [E050] Can't find model 'ru_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

## Часть 6: Работа с русскими текстами
Добавлена **лемматизация** для русского языка с использованием `spaCy`.
